In [1]:
#import dependencies
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from sqlite_functions_test import *
import requests
import glob
import os
import re

database = "market_data.db"

# CoinGecko API Pull

In [ ]:
# Set the CoinGecko API URL for historical data
# https://www.coingecko.com/en/api/documentation

url = 'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart'

# Define a function to fetch historical data for a given cryptocurrency
# Pulls price, market cap, and total volume data and appends to a dataframe
def fetch_crypto_data(days, coin_id, interval='daily'):
    parameters = {
        'vs_currency': 'usd',
        'days': days,
        'interval': interval
    }
    
    response = requests.get(url.format(coin_id=coin_id), params=parameters)
    data = response.json()
    
    if response.status_code == 200:
        df_prices = pd.DataFrame(data['prices'], columns=['timestamp', f'{coin_id}_Price'])
        df_market_caps = pd.DataFrame(data['market_caps'], columns=['timestamp', f'{coin_id}_Market_Cap'])
        df_total_volumes = pd.DataFrame(data['total_volumes'], columns=['timestamp', f'{coin_id}_Volume'])
        
        df = pd.merge(df_prices, df_market_caps, on='timestamp', how='outer')
        df = pd.merge(df, df_total_volumes, on='timestamp', how='outer')
        
        df['Date'] = pd.to_datetime(df['timestamp'], unit='ms')
        #df['date'] = df['timestamp'].dt.strftime('%m/%d/%Y')
        df.set_index('Date', inplace=True)
        df.drop(columns=['timestamp'], inplace=True)
        
        return df
    else:
        print(f"Error: {response.status_code}")
        return None

In [ ]:
# Fetch historical data for Bitcoin (BTC) and Dogecoin (DOGE)
num_days = '1200' # 1/1/2020 to 04/13/2023
coins = ['binancecoin','bitcoin', 'cardano', 'dogecoin', 'ethereum', 'polkadot', 'ripple', 'solana', 'uniswap']
today = datetime.now().strftime('%m/%d/%Y')

merged_data = None

for coin in coins:
    coin_df = fetch_crypto_data(num_days, coin_id=coin)
    
    if merged_data is None:
        merged_data = coin_df
    else:
        merged_data = pd.merge(merged_data, coin_df, on='Date', how='outer')

# Display the merged data
merged_data.dropna(how='any',axis=0, inplace=True) 
merged_data = merged_data[merged_data.index != today]

merged_data


In [ ]:
#convert wide data to long data
long_data = merged_data.reset_index()

# Convert using the melt function
long_data = long_data.melt(id_vars='Date', var_name='coin_metric', value_name='value')

# Extract the coin and metric names from the 'coin_metric' column
coin_metric_pattern = re.compile(r'(.+?)_(.+)')
long_data[['Coin', 'metric']] = long_data['coin_metric'].str.extract(coin_metric_pattern)

# Drop coin_metric column
long_data.drop(columns=['coin_metric'], inplace=True)

# Pivot the long format data
pivoted_data = long_data.pivot_table(index=['Date', 'Coin'], columns='metric', values='value').reset_index()

# Rename the columns
pivoted_data.columns.name = None

# Reorder the columns to the desired output format
pivoted_data = pivoted_data[['Date', 'Coin', 'Price', 'Volume', 'Market_Cap']]

# pivoted_data = pivoted_data.rename(columns={
#     'date': 'Date',
#     'coin': 'Coin',
#     'price': 'Price',
#     'total_volume': 'Volume',
#     'market_cap': 'Market_Cap'
# })

# save as Unix time for compatibility with Tableau??
#pivoted_data['date'] = pd.to_datetime(pivoted_data['date'], format='%m/%d/%Y')
#pivoted_data['date'] = pivoted_data['date'].apply(lambda x: int(x.timestamp()))


pivoted_data

In [ ]:
conn = create_connection(database)

# Insert the DataFrame into the 'market_data_plz' table
pivoted_data.to_sql('Crypto_Data', conn, if_exists='replace', index=False)

conn.close()

# yFinance Market Data PULL

In [2]:
## Ticker List and Start date
## why take out VIX and Banking Sector?
tickers = ["BTC-USD", "ETH-USD", "GLD", "SLV", "CL=F", "VIXY", "^IXIC", "^GSPC", "^DJI", "META", "AMZN", "AAPL", "NFLX", "GOOG", "TSLA", "JPM", "WFC", "C", "BAC", "UUP", "IEF"]
start_date = "2020-01-01"

# Fetch historical data
market_data = yf.download(tickers, start=start_date)

# Dataframe preview
market_data.head()

[*********************100%***********************]  21 of 21 completed


Adj Close                                                  \
                  AAPL        AMZN        BAC       BTC-USD          C   
Date                                                                     
2021-01-01         NaN         NaN        NaN  29374.152344        NaN   
2021-01-02         NaN         NaN        NaN  32127.267578        NaN   
2021-01-03         NaN         NaN        NaN  32782.023438        NaN   
2021-01-04  127.679985  159.331497  28.606859  31971.914062  55.554749   
2021-01-05  129.258591  160.925507  28.825958  33992.429688  56.995815   

                                                                       ...  \
                 CL=F      ETH-USD         GLD       GOOG         IEF  ...   
Date                                                                   ...   
2021-01-01        NaN   730.367554         NaN        NaN         NaN  ...   
2021-01-02        NaN   774.534973         NaN        NaN         NaN  ...   
2021-01-03        NaN   975.507690         NaN        NaN         NaN  ...   
2021-01-04  47.619999  1040.233032  182.330002  86.412003  116.085579  ...   
2021-01-05  49.930000  1100.006104  182.869995  87.045998  115.814659  ...   

                Volume                                                 \
                  META       NFLX         SLV         TSLA        UUP   
Date                                                                    
2021-01-01         NaN        NaN         NaN          NaN        NaN   
2021-01-02         NaN        NaN         NaN          NaN        NaN   
2021-01-03         NaN        NaN         NaN          NaN        NaN   
2021-01-04  15106100.0  4444400.0  54026000.0  145914600.0  1322800.0   
2021-01-05   9871600.0  3133900.0  32882000.0   96735600.0   723300.0   

                                                                            
                 VIXY         WFC         ^DJI         ^GSPC         ^IXIC  
Date                                                                        
2021-01-01        NaN         NaN          NaN           NaN           NaN  
2021-01-02        NaN         NaN          NaN           NaN           NaN  
2021-01-03        NaN         NaN          NaN           NaN           NaN  
2021-01-04  3254100.0  31421600.0  475080000.0  5.015000e+09  6.636170e+09  
2021-01-05  2013825.0  37360000.0  350910000.0  4.591020e+09  6.971860e+09  

[5 rows x 126 columns]

In [3]:
#Adjusted close is the closing price after adjustments for all applicable splits and dividend distributions. 
#Data is adjusted using appropriate split and dividend multipliers, adhering to Center for Research in Security Prices (CRSP) standards.

#drop multi-index column header
market_data.columns = [f'{col[1]}_{col[0]}' for col in market_data.columns]

market_data_df = market_data.iloc[:, np.r_[:21, 105:126]]
market_data_df = market_data_df.fillna(method='ffill', inplace=False)
market_data_df

,AAPL_Adj Close,AMZN_Adj Close,BAC_Adj Close,BTC-USD_Adj Close,C_Adj Close,CL=F_Adj Close,ETH-USD_Adj Close,GLD_Adj Close,GOOG_Adj Close,IEF_Adj Close,...,META_Volume,NFLX_Volume,SLV_Volume,TSLA_Volume,UUP_Volume,VIXY_Volume,WFC_Volume,^DJI_Volume,^GSPC_Volume,^IXIC_Volume
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-01,NaN,NaN,NaN,29374.152344,NaN,NaN,730.367554,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-02,NaN,NaN,NaN,32127.267578,NaN,NaN,774.534973,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-03,NaN,NaN,NaN,32782.023438,NaN,NaN,975.507690,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-04,127.679985,159.331497,28.606859,31971.914062,55.554749,47.619999,1040.233032,182.330002,86.412003,116.085579,...,15106100.0,4444400.0,54026000.0,145914600.0,1322800.0,3254100.0,31421600.0,475080000.0,5.015000e+09,6.636170e+09
2021-01-05,129.258591,160.925507,28.825958,33992.429688,56.995815,49.930000,1100.006104,182.869995,87.045998,115.814659,...,9871600.0,3133900.0,32882000.0,96735600.0,723300.0,2013825.0,37360000.0,350910000.0,4.591020e+09,6.971860e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-11,160.800003,99.919998,28.709999,30235.058594,47.200001,81.529999,1892.189697,186.279999,106.120003,99.449997,...,16710100.0,4044800.0,14526500.0,115770900.0,785100.0,3532300.0,25116100.0,241430000.0,3.665830e+09,4.676960e+09
2023-04-12,160.100006,97.830002,28.480000,30139.052734,46.919998,83.260002,1920.682129,187.190002,105.220001,99.739998,...,18972200.0,3965400.0,21592400.0,150256300.0,2645600.0,8530000.0,20252900.0,260300000.0,3.633120e+09,4.909770e+09
2023-04-13,165.559998,102.400002,28.559999,30399.066406,47.299999,82.160004,2012.634644,189.720001,108.190002,99.410004,...,23310400.0,7406400.0,30298700.0,112933000.0,2468300.0,6313400.0,22397800.0,277230000.0,3.596590e+09,4.942990e+09


In [4]:
# Create a connection to the SQLite database
conn = create_connection(database)

# Insert the historical data into the SQLite database
market_data_df.to_sql("economy_data", conn, if_exists="replace", index=True)

# Commit the changes and close the connection
conn.commit()
conn.close()

In [5]:
# Create a connection to the SQLite database
conn = create_connection(database)

# Check if the connection is successful
if conn is not None:
    # Read the data from the SQLite database into a Pandas DataFrame
    market_data = pd.read_sql_query("SELECT * FROM economy_data", conn)

    # Close the connection to the SQLite database
    conn.close()
else:
    print("Error! Cannot create the database connection.")

market_data

,Date,AAPL_Adj Close,AMZN_Adj Close,BAC_Adj Close,BTC-USD_Adj Close,C_Adj Close,CL=F_Adj Close,ETH-USD_Adj Close,GLD_Adj Close,GOOG_Adj Close,...,META_Volume,NFLX_Volume,SLV_Volume,TSLA_Volume,UUP_Volume,VIXY_Volume,WFC_Volume,^DJI_Volume,^GSPC_Volume,^IXIC_Volume
0,2021-01-01 00:00:00,NaN,NaN,NaN,29374.152344,NaN,NaN,730.367554,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-02 00:00:00,NaN,NaN,NaN,32127.267578,NaN,NaN,774.534973,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-03 00:00:00,NaN,NaN,NaN,32782.023438,NaN,NaN,975.507690,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-04 00:00:00,127.679985,159.331497,28.606859,31971.914062,55.554749,47.619999,1040.233032,182.330002,86.412003,...,15106100.0,4444400.0,54026000.0,145914600.0,1322800.0,3254100.0,31421600.0,475080000.0,5.015000e+09,6.636170e+09
4,2021-01-05 00:00:00,129.258591,160.925507,28.825958,33992.429688,56.995815,49.930000,1100.006104,182.869995,87.045998,...,9871600.0,3133900.0,32882000.0,96735600.0,723300.0,2013825.0,37360000.0,350910000.0,4.591020e+09,6.971860e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830,2023-04-11 00:00:00,160.800003,99.919998,28.709999,30235.058594,47.200001,81.529999,1892.189697,186.279999,106.120003,...,16710100.0,4044800.0,14526500.0,115770900.0,785100.0,3532300.0,25116100.0,241430000.0,3.665830e+09,4.676960e+09
831,2023-04-12 00:00:00,160.100006,97.830002,28.480000,30139.052734,46.919998,83.260002,1920.682129,187.190002,105.220001,...,18972200.0,3965400.0,21592400.0,150256300.0,2645600.0,8530000.0,20252900.0,260300000.0,3.633120e+09,4.909770e+09
832,2023-04-13 00:00:00,165.559998,102.400002,28.559999,30399.066406,47.299999,82.160004,2012.634644,189.720001,108.190002,...,23310400.0,7406400.0,30298700.0,112933000.0,2468300.0,6313400.0,22397800.0,277230000.0,3.596590e+09,4.942990e+09
833,2023-04-14 00:00:00,165.210007,102.510002,29.520000,30399.066406,49.560001,82.519997,2012.634644,186.360001,109.459999,...,21574200.0,5345600.0,34942400.0,96306500.0,2777400.0,5018500.0,40936900.0,276720000.0,3.575690e+09,4.655740e+09


# yFinance stock and market additional info

In [6]:
#  Ability to access additoinal information about the stock through yf.Ticker().info
market_info_df = pd.DataFrame()


for stock in tickers: 
    info = yf.Ticker(stock).info
    print(info)
    marketcap = info.get('marketCap', None)
    shortname = info.get('shortName', None)   
    longname = info.get('longName', None)    
    market_info_df = market_info_df.append({'Stock':stock,'marketcap':marketcap, 'shortname':shortname}, ignore_index=True)
    
print(market_info_df)

{'symbol': 'BTC-USD', 'twoHundredDayAverageChangePercent': 0.4461606, 'fiftyTwoWeekLowChangePercent': 0.9430449, 'language': 'en-US', 'circulatingSupply': 19347288, 'regularMarketDayRange': '30253.377 - 30573.51', 'regularMarketDayHigh': 30573.51, 'twoHundredDayAverageChange': 9350.947, 'fromCurrency': 'BTC', 'twoHundredDayAverage': 20958.701, 'marketCap': 586409508864, 'fiftyTwoWeekHighChange': -12583.934, 'fiftyTwoWeekRange': '15599.047 - 42893.582', 'lastMarket': 'CoinMarketCap', 'fiftyDayAverageChange': 4457.6484, 'firstTradeDateMilliseconds': 1410912000000, 'exchangeDataDelayedBy': 0, 'averageDailyVolume3Month': 25233900541, 'logoUrl': 'https://s2.coinmarketcap.com/static/img/coins/64x64/1.png', 'fiftyTwoWeekLow': 15599.047, 'market': 'ccc_market', 'regularMarketVolume': 12197079040, 'quoteSourceName': 'CoinMarketCap', 'messageBoardId': 'finmb_BTC_CCC', 'toCurrency': 'USD=X', 'priceHint': 2, 'regularMarketDayLow': 30253.377, 'sourceInterval': 15, 'exchange': 'CCC', 'shortName': 'B

C:\Users\zarbi\AppData\Local\Temp\ipykernel_42080\4265765679.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  market_info_df = market_info_df.append({'Stock':stock,'marketcap':marketcap, 'shortname':shortname}, ignore_index=True)


{'symbol': 'ETH-USD', 'twoHundredDayAverageChangePercent': 0.42600513, 'fiftyTwoWeekLowChangePercent': 1.336354, 'language': 'en-US', 'circulatingSupply': 120109688, 'regularMarketDayRange': '2077.695 - 2108.9333', 'regularMarketDayHigh': 2108.9333, 'twoHundredDayAverageChange': 625.4509, 'fromCurrency': 'ETH', 'twoHundredDayAverage': 1468.177, 'fiftyTwoWeekHighChange': -1079.8235, 'marketCap': 251464990720, 'fiftyTwoWeekRange': '896.109 - 3173.4514', 'lastMarket': 'CoinMarketCap', 'fiftyDayAverageChange': 372.51514, 'firstTradeDateMilliseconds': 1510185600000, 'exchangeDataDelayedBy': 0, 'averageDailyVolume3Month': 8736691435, 'logoUrl': 'https://s2.coinmarketcap.com/static/img/coins/64x64/1027.png', 'fiftyTwoWeekLow': 896.109, 'regularMarketVolume': 8123575296, 'market': 'ccc_market', 'quoteSourceName': 'CoinMarketCap', 'messageBoardId': 'finmb_ETH_CCC', 'toCurrency': 'USD=X', 'priceHint': 2, 'sourceInterval': 15, 'exchange': 'CCC', 'regularMarketDayLow': 2077.695, 'region': 'US', 's

C:\Users\zarbi\AppData\Local\Temp\ipykernel_42080\4265765679.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  market_info_df = market_info_df.append({'Stock':stock,'marketcap':marketcap, 'shortname':shortname}, ignore_index=True)


{'symbol': 'GLD', 'twoHundredDayAverageChangePercent': 0.11698558, 'trailingThreeMonthNavReturns': 4.25151, 'netAssets': 59048718000.0, 'fiftyTwoWeekLowChangePercent': 0.23769669, 'language': 'en-US', 'regularMarketDayRange': '185.17 - 188.48', 'regularMarketDayHigh': 188.48, 'twoHundredDayAverageChange': 19.518097, 'twoHundredDayAverage': 166.8419, 'askSize': 9, 'bookValue': 170.017, 'marketCap': 48509509632, 'fiftyTwoWeekHighChange': -4.050003, 'fiftyTwoWeekRange': '150.57 - 190.41', 'fiftyDayAverageChange': 8.944, 'exchangeDataDelayedBy': 0, 'firstTradeDateMilliseconds': 1100788200000, 'averageDailyVolume3Month': 7921748, 'trailingAnnualDividendRate': 0.0, 'fiftyTwoWeekLow': 150.57, 'regularMarketVolume': 13272541, 'market': 'us_market', 'postMarketPrice': 186.46, 'quoteSourceName': 'Delayed Quote', 'messageBoardId': 'finmb_5600649', 'priceHint': 2, 'sourceInterval': 15, 'regularMarketDayLow': 185.17, 'ytdReturn': 9.85617, 'exchange': 'PCX', 'region': 'US', 'shortName': 'SPDR Gold T

...

C:\Users\zarbi\AppData\Local\Temp\ipykernel_42080\4265765679.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  market_info_df = market_info_df.append({'Stock':stock,'marketcap':marketcap, 'shortname':shortname}, ignore_index=True)


{'symbol': 'UUP', 'twoHundredDayAverageChangePercent': -0.03349931, 'trailingThreeMonthNavReturns': 0.76447, 'netAssets': 1131942140.0, 'fiftyTwoWeekLowChangePercent': 0.034766365, 'language': 'en-US', 'regularMarketDayRange': '27.565 - 27.72', 'regularMarketDayHigh': 27.72, 'twoHundredDayAverageChange': -0.9594002, 'askSize': 9, 'twoHundredDayAverage': 28.6394, 'fiftyTwoWeekHighChange': -3.08, 'fiftyTwoWeekRange': '26.75 - 30.76', 'fiftyDayAverageChange': -0.35879898, 'firstTradeDateMilliseconds': 1172759400000, 'exchangeDataDelayedBy': 0, 'averageDailyVolume3Month': 3239858, 'trailingAnnualDividendRate': 0.0, 'fiftyTwoWeekLow': 26.75, 'market': 'us_market', 'regularMarketVolume': 2777446, 'postMarketPrice': 27.67, 'quoteSourceName': 'Nasdaq Real Time Price', 'messageBoardId': 'finmb_31289611', 'priceHint': 2, 'exchange': 'PCX', 'sourceInterval': 15, 'regularMarketDayLow': 27.565, 'ytdReturn': -0.46746, 'region': 'US', 'shortName': 'Invesco DB USD Index Bullish Fu', 'fiftyDayAverageCh

C:\Users\zarbi\AppData\Local\Temp\ipykernel_42080\4265765679.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  market_info_df = market_info_df.append({'Stock':stock,'marketcap':marketcap, 'shortname':shortname}, ignore_index=True)


{'symbol': 'IEF', 'twoHundredDayAverageChangePercent': 0.0039795577, 'trailingThreeMonthNavReturns': 0.54707, 'netAssets': 29834223600.0, 'fiftyTwoWeekLowChangePercent': 0.06963662, 'language': 'en-US', 'regularMarketDayRange': '98.79 - 99.1', 'regularMarketDayHigh': 99.1, 'twoHundredDayAverageChange': 0.39209747, 'askSize': 22, 'twoHundredDayAverage': 98.5279, 'bookValue': 117.158, 'fiftyTwoWeekHighChange': -6.830002, 'marketCap': 14442319872, 'fiftyTwoWeekRange': '92.48 - 105.75', 'fiftyDayAverageChange': 1.2483978, 'firstTradeDateMilliseconds': 1028035800000, 'exchangeDataDelayedBy': 0, 'averageDailyVolume3Month': 9774172, 'trailingAnnualDividendRate': 0.934, 'fiftyTwoWeekLow': 92.48, 'market': 'us_market', 'regularMarketVolume': 4968542, 'postMarketPrice': 98.9, 'quoteSourceName': 'Delayed Quote', 'messageBoardId': 'finmb_8797472', 'priceHint': 2, 'exchange': 'NGM', 'sourceInterval': 15, 'regularMarketDayLow': 98.79, 'ytdReturn': 3.91683, 'region': 'US', 'shortName': 'iShares 7-10 

C:\Users\zarbi\AppData\Local\Temp\ipykernel_42080\4265765679.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  market_info_df = market_info_df.append({'Stock':stock,'marketcap':marketcap, 'shortname':shortname}, ignore_index=True)


In [7]:
# Add custom labels to the DataFrame based on the ticker symbol
labels = ['Crypto', 'Crypto', 'Commodities', 'Commodities', 'Commodities', 'Volatility', 'Indexes', 'Indexes', 'Indexes', 'Tech', 'Tech', 'Tech', 'Tech', 'Tech', 'Tech', 'Finance', 'Finance', 'Finance', 'Finance', 'Currency', 'Gov. Bonds']
market_info_df['Market Grouping'] = labels[:len(market_info_df)]


In [8]:
print(market_info_df.shape, market_data_df.shape)

(21, 4) (835, 42)


In [9]:
# Create a connection to the SQLite database

conn = create_connection(database)

# Insert the historical data into the SQLite database
market_info_df.to_sql("market_info", conn, if_exists="replace", index=True)

# Commit the changes and close the connection
conn.commit()
conn.close()

# Additional data CSV Imports

In [23]:
# Find all .csv and .xlsx files
path = os.getcwd()
file_list = glob.glob(os.path.join(path, "../../Production/Resources/*.csv")) + \
            glob.glob(os.path.join(path, "../../Production/Resources/*.xlsx"))

dict_of_df = {} # initialize empty dictionary
for file in file_list:

    filename = os.path.splitext(os.path.basename(file))[0]

    if file.endswith('.csv'):
        print('Location:', file)
        df = pd.read_csv(file)
        print('Loaded file:', filename)
    elif file.endswith('.xlsx'):
        print('Location:', file)
        df = pd.read_excel(file, engine='openpyxl')
        print('Loaded file:', filename)

    # Replace dashes and asterisks with NaN values
    df.replace(['-','--', '*'], [None, None, None], inplace=True)
    print(df.head())

    dict_of_df[filename] = df


Location: e:\Thins\School\UTAustin Bootcamp\UTA_Project-4\sandbox\JustinB\../../Production/Resources\DP_LIVE_10042023003622990.csv
Loaded file: DP_LIVE_10042023003622990
  LOCATION INDICATOR SUBJECT MEASURE FREQUENCY  TIME      Value  Flag Codes
0      AUS  TRUSTGOV     TOT      PC         A  2006  53.206333         NaN
1      AUS  TRUSTGOV     TOT      PC         A  2007  52.693281         NaN
2      AUS  TRUSTGOV     TOT      PC         A  2008  64.467542         NaN
3      AUS  TRUSTGOV     TOT      PC         A  2010  60.975292         NaN
4      AUS  TRUSTGOV     TOT      PC         A  2011  53.078672         NaN
Location: e:\Thins\School\UTAustin Bootcamp\UTA_Project-4\sandbox\JustinB\../../Production/Resources\tbmics.csv
Loaded file: tbmics
      Month  YYYY  ICS_ALL
0  November  1952     86.2
1  February  1953     90.7
2    August  1953     80.8
3  November  1953     80.7
4  February  1954     82.0
Location: e:\Thins\School\UTAustin Bootcamp\UTA_Project-4\sandbox\JustinB\../../

In [21]:
dict_of_df

{'cpi':      Year Month  CPI
 0    2013   Jan  1.6
 1    2013   Feb  2.0
 2    2013   Mar  1.5
 3    2013   Apr  1.1
 4    2013   May  1.4
 ..    ...   ...  ...
 118  2022   Nov  7.1
 119  2022   Dec  6.5
 120  2023   Jan  6.4
 121  2023   Feb  6.0
 122  2023   Mar  5.0
 
 [123 rows x 3 columns],
 'gallup_banks':         Year  Great deal  Quite a lot  Some  Very little None (vol.)  \
 0       2022           9           18    50           22           1   
 1       2021          12           21    42           23           1   
 2       2020          17           21    43           18           1   
 3       2019          13           17    43           25           1   
 4       2018          13           17    46           21           1   
 5       2017          14           18    39           27           1   
 6       2016          11           16    47           25           1   
 7       2015          12           16    45           24           2   
 8       2014          10    

In [24]:
dict_of_df['gallup_police']

###Haivng Prolbems with None (vol.) import?? Need to fix.  Is it because of column name? 

,Year,Great deal,Quite a lot,Some,Very little,None (vol.),No opinion,Great deal/Quite a lot
0,2022,19,26,37,17,1,None,45
1,2021,26,25,32,16,1,None,51
2,2020,23,25,33,17,2,None,48
3,2019,29,24,31,16,1,None,53
4,2018,29,25,31,14,1,1,54
5,2017,31,26,28,14,1,1,57
6,2016,25,31,29,13,1,1,56
7,2015,25,27,30,16,2,1,52
8,2014,25,28,31,14,2,None,53
9,2013,26,31,30,12,1,1,57


In [25]:
# Create a connection to the SQLite database
conn = create_connection(database)

if conn is not None:
    for table_name, df in dict_of_df.items():
        # Replace any special characters or spaces in the table_name with an underscore
        table_name = ''.join(e if e.isalnum() else '_' for e in table_name)
        
        # Insert the DataFrame into the table
        df.to_sql(table_name, conn, if_exists='replace', index=False)
else:
    print("Error! Cannot create the database connection.")

# Commit the changes and close the connection
conn.commit()
conn.close()